In [1]:
pwd

'/opt/ml/final-project-level3-cv-02/airflow/dags'

In [2]:
cd /opt/ml/final-project-level3-cv-02/airflow

/opt/ml/final-project-level3-cv-02/airflow


In [18]:
import pendulum
from datetime import time, timedelta, datetime

from airflow import DAG 
from airflow.utils.dates import days_ago
from airflow.operators.bash import BashOperator
from airflow.providers.mysql.operators.mysql import MySqlOperator
from airflow.operators.python import PythonOperator
from airflow.models.variable import Variable

import os
import pymysql
import pandas as pd
from sqlalchemy import create_engine
from collections import defaultdict

# .env
from dotenv import load_dotenv
load_dotenv(dotenv_path = "../../.env")
MYSQL_USER = os.getenv('MYSQL_USER')
MYSQL_PASSWORD = os.getenv('MYSQL_PASSWORD')
MYSQL_PORT = os.getenv('MYSQL_PORT')
MYSQL_DBNAME = os.getenv('MYSQL_DB')
MYSQL_SERVER = os.getenv('MYSQL_SERVER')

In [15]:
print(MYSQL_USER)

cv02


In [8]:
!ls

__pycache__    hello_world.py  scripts	   user_statics.py
connect_db.py  inference.py    test.ipynb


In [9]:
!ls ../..

age_gender_classifier  airflow.db  back-end  front-end	sql_app
airflow		       babygan	   docker    logs


In [68]:
AIRFLOW_HOME = ' /opt/ml/final-project-level3-cv-02/airflow'
INFER_RESULT_PATH = os.path.join(AIRFLOW_HOME, 'infer_result')

def connect_db_get_data():
    MYSQL_USER = os.getenv('MYSQL_USER')
    MYSQL_PASSWORD = os.getenv('MYSQL_PASSWORD')
    MYSQL_PORT = os.getenv('MYSQL_PORT')
    MYSQL_DBNAME = os.getenv('MYSQL_DBNAME')
    MYSQL_SERVER = os.getenv('MYSQL_SERVER')

    print(MYSQL_USER, MYSQL_PORT, type(MYSQL_PORT))

    DB = pymysql.connect(host=MYSQL_SERVER, user=MYSQL_USER, passwd=MYSQL_PASSWORD, port=int(MYSQL_PORT), charset='utf8')

    data = pd.read_sql('''
    select * from cv02.inference_result''', con = DB)
    print(data.shape[0])

    os.makedirs(INFER_RESULT_PATH, exist_ok=True)
    data.to_csv(f'{INFER_RESULT_PATH}/infer_result_latest.csv', index=False)
    print(f'save inference_result_latest in {INFER_RESULT_PATH}')
    return data
   

def calculate_bounce_rate(data):
    print(f'calculate bounce rate after click inference button')
    df_log = pd.read_csv(f'{INFER_RESULT_PATH}/infer_result_latest.csv', parse_dates = ['created_time'])
    df_log['age'] = df_log['age'].fillna(0)
    df_log['gender'] = df_log['gender'].fillna('male')

    df_log['age'] = df_log['age'].astype(int)
    total_id = len(df_log)
    df_statics = pd.DataFrame(columns = ['ID', 'age', 'gender', 'inference_bounce_rate'])

    # calculate inference_bounce_rate
    # total_b_rate = len(df_log['complete' == False]) / len(df_log) # inference_bounce_rate -> #bounce / #click_inference_button = #complete==False / #ID
    df_statics['ID'] = [i for i in range(1,11)]

    # gender - age 
    gen_age = defaultdict(str)
    gender = ['male', 'female']
    age = [10, 20,30,40,50, 100]
    gen_age['female'] = []
    gen_age['male'] = []

    total_data = []
    id = 1
    for g in gender:
        for a_i in range(len(age)-1):
            condition = (df_log.gender == g) & ((int(age[a_i]) <= df_log["age"]) & ( df_log["age"] < int(age[a_i+1])))
            rate = len(df_log[condition]) / total_id * 100
            total_data.append([id, g, age[a_i], rate])
            # gen_age[g].append(len(df_log[condition]))
            id+=1

    df_result = pd.DataFrame(total_data, columns=['id', 'gender', 'age', 'rate'])
    data_to_db(df_result, 'user_statistic')
    
    return

def data_to_db(df_result, table_name):
    DB = pymysql.connect(host=MYSQL_SERVER, user=MYSQL_USER, passwd=MYSQL_PASSWORD, port=int(MYSQL_PORT), charset='utf8')

    engine = create_engine(f'mysql+pymysql://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_SERVER}:{MYSQL_PORT}/{MYSQL_DBNAME}?charset=utf8mb4')
    engine_conn = engine.connect()
    df_result.to_sql(table_name , engine_conn, if_exists='replace', index=None)
    engine_conn.close()
    engine.dispose()

In [69]:
data = connect_db_get_data()

cv02 3306 <class 'str'>
8
save inference_result_latest in  /opt/ml/final-project-level3-cv-02/airflow/infer_result


In [70]:
data

,id,father_url,mother_url,baby_url,created_time,comment,closed_at,complete,age,gender
0,013f5fc1-a,https://12war.s3.ap-northeast-2.amazonaws.com/...,https://12war.s3.ap-northeast-2.amazonaws.com/...,baby_url_test,2021-12-19 10:04:36,dd,2021-12-19 11:02:26,None,None,None
1,123,strin,strin,stri111n,NaT,None,2021-12-19 11:02:26,None,None,None
2,24203a37-7,https://12war.s3.ap-northeast-2.amazonaws.com/...,https://12war.s3.ap-northeast-2.amazonaws.com/...,baby_url_test,2021-12-19 10:05:24,dd,2021-12-19 11:02:26,None,None,None
3,6ae3a923-e,https://12war.s3.ap-northeast-2.amazonaws.com/...,https://12war.s3.ap-northeast-2.amazonaws.com/...,baby_url_test,2021-12-19 10:00:08,dd,2021-12-19 11:02:26,None,None,None
4,a1d83841-5,https://12war.s3.ap-northeast-2.amazonaws.com/...,https://12war.s3.ap-northeast-2.amazonaws.com/...,baby_url_test,2021-12-19 10:05:43,dd,2021-12-19 11:02:26,None,None,None
5,ss2,string,string,string,2021-12-19 08:03:22,good,2021-12-19 11:02:26,None,None,None
6,strinfffg,string,string,string,2021-12-19 09:47:45,string,2021-12-19 11:02:26,None,None,None
7,string,string,string,string,2021-12-19 08:36:34,string,2021-12-19 11:02:26,None,None,None


In [71]:
data = data.append({'id':'asd', 'father_url':'asd', 'mother_url':'df', 'baby_url':'asff', 'created_time':datetime.now(), 'comment':'com1', 'closed_at': datetime.now(), 'complete':False,'age': 20, 'gender':'female'}, ignore_index=True)
data = data.append({'id':'asdf', 'father_url':'aswd', 'mother_url':'awdf', 'baby_url':'asff', 'created_time':datetime.now(), 'comment':'com2', 'closed_at': datetime.now(), 'complete':True,'age': 30, 'gender':'male'}, ignore_index=True)
data = data.append({'id':'assd', 'father_url':'absd', 'mother_url':'dfs', 'baby_url':'asff', 'created_time':datetime.now(), 'comment':'com3', 'closed_at': datetime.now(), 'complete':False,'age': 40, 'gender':'female'}, ignore_index=True)


data


,id,father_url,mother_url,baby_url,created_time,comment,closed_at,complete,age,gender
0,013f5fc1-a,https://12war.s3.ap-northeast-2.amazonaws.com/...,https://12war.s3.ap-northeast-2.amazonaws.com/...,baby_url_test,2021-12-19 10:04:36.000000,dd,2021-12-19 11:02:26.000000,None,None,None
1,123,strin,strin,stri111n,NaT,None,2021-12-19 11:02:26.000000,None,None,None
2,24203a37-7,https://12war.s3.ap-northeast-2.amazonaws.com/...,https://12war.s3.ap-northeast-2.amazonaws.com/...,baby_url_test,2021-12-19 10:05:24.000000,dd,2021-12-19 11:02:26.000000,None,None,None
3,6ae3a923-e,https://12war.s3.ap-northeast-2.amazonaws.com/...,https://12war.s3.ap-northeast-2.amazonaws.com/...,baby_url_test,2021-12-19 10:00:08.000000,dd,2021-12-19 11:02:26.000000,None,None,None
4,a1d83841-5,https://12war.s3.ap-northeast-2.amazonaws.com/...,https://12war.s3.ap-northeast-2.amazonaws.com/...,baby_url_test,2021-12-19 10:05:43.000000,dd,2021-12-19 11:02:26.000000,None,None,None
5,ss2,string,string,string,2021-12-19 08:03:22.000000,good,2021-12-19 11:02:26.000000,None,None,None
6,strinfffg,string,string,string,2021-12-19 09:47:45.000000,string,2021-12-19 11:02:26.000000,None,None,None
7,string,string,string,string,2021-12-19 08:36:34.000000,string,2021-12-19 11:02:26.000000,None,None,None
8,asd,asd,df,asff,2021-12-19 12:50:14.364571,com1,2021-12-19 12:50:14.364574,False,20,female
9,asdf,aswd,awdf,asff,2021-12-19 12:50:14.371278,com2,2021-12-19 12:50:14.371281,True,30,male


In [72]:
gen_age = calculate_bounce_rate(data)
gen_age

calculate bounce rate after click inference button
0
0
0
0
0
0
0
0
0
0
[[1, 'male', 10, 0.0], [2, 'male', 20, 0.0], [3, 'male', 30, 0.0], [4, 'male', 40, 0.0], [5, 'male', 50, 0.0], [6, 'female', 10, 0.0], [7, 'female', 20, 0.0], [8, 'female', 30, 0.0], [9, 'female', 40, 0.0], [10, 'female', 50, 0.0]]


[[1, 'male', 10, 0.0],
 [2, 'male', 20, 0.0],
 [3, 'male', 30, 0.0],
 [4, 'male', 40, 0.0],
 [5, 'male', 50, 0.0],
 [6, 'female', 10, 0.0],
 [7, 'female', 20, 0.0],
 [8, 'female', 30, 0.0],
 [9, 'female', 40, 0.0],
 [10, 'female', 50, 0.0]]

In [8]:
import pymysql

connect = pymysql.connect(
			user=os.getenv('MYSQL_USER'), 
			passwd=os.getenv('MYSQL_PASSWORD'), 
			host=os.getenv('MYSQL_SERVER'),
			db=os.getenv('MYSQL_DB'), 
			charset='utf8'
		)
cursor = connect.cursor()

In [10]:
connect.close()
# connect.dispose()

Error: Already closed